# STI 2018 - DOIs, URLs, and FB

Code to produce quantification of three problem cases with the WOS state_of_oa dataset

In [33]:
import pandas as pd
from pprint import pprint
import json
from tqdm import tqdm

import collections
import numpy as np
import itertools
from urllib.parse import urlparse

In [91]:
pd.options.display.float_format = '{:,.1f}'.format

In [36]:
# Load resolved DOIs
resolved_doi = pd.read_csv("data/wos_100k_resolved.csv", index_col="doi")
resolved_doi['domain'] = resolved_doi.resolved.map(lambda x: urlparse(x)[1] if pd.notnull(x) else None)
resolved_doi['prefix'] = resolved_doi.index.map(lambda x: x.split("/")[0])

# Challenge 1

## Breakdown of responses

In [94]:
df = resolved_doi

l = len(df)
print("Total number of articles: {}".format(l))

a = df.status_code.notnull().sum()
print("Got response from crossref: {} ({:.1f}%)".format(a, a*100/l))
a = df.status_code.value_counts()[200]
print("Resolved with 200s: {} ({:.1f}%)".format(a, a*100/l))
a = len(df[df.status_code.notnull()]) - a
print("Resolved with error: {} ({:.1f}%)".format(a, a*100/l))
a = df.err.value_counts()['RequestException'] + df.err.value_counts()['Timeout']
print("RequestException + TimeOuts: {} ({:.1f}%)".format(a, a*100/l))

https_urls = df[df.resolved.notnull()].resolved.map(lambda x: x[4] == "s").sum()
print("Resolved to HTTPS: {}\tHTTP: {}".format(https_urls, len(df[df.resolved.notnull()])-https_urls))

dupl = df.resolved.duplicated(keep=False)
notnull = df.resolved.notnull()
print("{} ({:.1f}%)".format(len(df[dupl&notnull]), len(df[dupl&notnull])/len(df[notnull])*100))
len(df[dupl&notnull])/l*100

Total number of articles: 103539
Got response from crossref: 91490 (88.4%)
Resolved with 200s: 85515 (82.6%)
Resolved with error: 5975 (5.8%)
RequestException + TimeOuts: 12049 (11.6%)
Resolved to HTTPS: 69619	HTTP: 21871
68 (0.1%)


0.06567573571311294

## How many DOIs resolve to HTTP/HTTPS

## URL Variations

In [71]:
big_5 = ['linkinghub.elsevier.com',
         'link.springer.com',
         'onlinelibrary.wiley.com',
         'www.tandfonline.com',
         'journals.sagepub.com']

ind = []
for d in big_5:
    ind.append(df[df.domain == d].sample().index[0])
    
a = len(df[df.domain.isin(big_5)])
print("Big 5 DOIs: {} ({:.1f}%)".format(a, 100*a/l))

print("Samples")
df.loc[ind][['status_code', 'resolved', 'domain', 'prefix']].resolved.tolist()

Big 5 DOIs: 55777 (53.9%)
Samples


['https://linkinghub.elsevier.com/retrieve/pii/S1549963414003256',
 'https://link.springer.com/article/10.1007%2Fs10337-010-1883-4',
 'https://onlinelibrary.wiley.com/doi/abs/10.1002/spe.909',
 'https://www.tandfonline.com/doi/full/10.1179/1743282014Y.0000000119',
 'http://journals.sagepub.com/doi/10.1177/1010539513486919']

# Challenge 2

In [179]:
# Consts
ids = ['ogid'+str(i) for i in range (1,5)]
eng = ['eng'+str(i) for i in range (1,5)]
urls = ['url'+str(i) for i in range (1,5)]
# shares = ['shares1','shares2','shares3','shares4']

dtype={}
for i in range(1,5):
    dtype['url'+str(i)] = str
    dtype['og_eng'+str(i)] = str
    dtype['og_obj'+str(i)] = str
    dtype['og_err'+str(i)] = str

url_response = pd.read_csv("data/wos_100k_full.csv", index_col="doi", parse_dates=['ts'], dtype=dtype)

# Prepare results
results = url_response[urls].copy()
for i in range(1,5):
    results['ogid'+str(i)] = url_response['og_obj'+str(i)].map(lambda x: json.loads(x)['id'] if pd.notnull(x) else None)
    results['eng'+str(i)] = url_response['og_eng'+str(i)].map(lambda x: sum(json.loads(x).values()) if pd.notnull(x) else None)
    #results['shares'+str(i)] = df['og_eng'+str(i)].map(lambda x: json.loads(x)['share_count'] if pd.notnull(x) else None)

In [180]:
x = results[eng].apply(lambda x: sum(x) > 0, axis=1)
results_eng = results[x]

x = results[ids].apply(lambda x: x.notnull().sum() > 0, axis=1)
results_ids = results[x]

## HTTP/HTTPS URL breakdown for articles with OG object or Eng>0

In [202]:
def get_https_breakdown(df):
    http = 0
    https = 0
    df = df[['url1', 'url2','ogid1', 'ogid2']]
    for row in df.itertuples():
        if row[3]:
            if row[1][4] == "s":
                https = https + 1
            else:
                http = http + 1
        if row[4]:
            if row[2][4] == "s":
                https = https + 1
            else:
                http = http + 1
    return {'http':http, 'https':https}
pd.DataFrame({'IDS':get_https_breakdown(results_ids),
              'ENG':get_https_breakdown(results_eng)}, )

,ENG,IDS
http,3821,19901
https,684,1856


## Coverage of 4 URL variations

In [203]:
cov = results[ids].apply(lambda x: x.notnull().sum()).values
cov_eng = results[eng].apply(lambda x: sum(x>0)).values
pd.DataFrame({'IDS':cov,
              'IDS (%)':cov/(len(results)/100),
              'ENG':cov_eng,
              'ENG (%)': cov_eng/(len(results)/100)})

,ENG,ENG (%),IDS,IDS (%)
0,1426,1.6,8452,9.2
1,2458,2.7,13305,14.5
2,74,0.1,179,0.2
3,2612,2.9,10124,11.1


## Problem 2 - DOI shares spread across graph objects

In [370]:
tdf = results
print("Total number of queried articles: {}".format(len(tdf)))

# At least one OG_ID
min_one = tdf[ids].dropna(how="all")
min_two_bool = min_one.apply(lambda x: x.notnull().sum() > 1, axis=1)
min_two = min_one[min_two_bool]

# ============================

l = len(tdf)
a = len(min_one)
print("Articles w/ at least 1 OG_ID:     {} ({:.2f}%)".format(a, 100*a/l))
a = len(min_two)
print("Articles w/ at least 2 OG_IDs:    {} ({:.2f}%)".format(a, 100*a/l))

rows_with_mismatch_ids_null = min_two.apply(lambda x: len(set(x)) != 1, axis=1)
rows_with_mismatch_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) != 1, axis=1)
rows_with_match_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 1 and len([y for y in x if pd.notnull(y)])>1, axis=1)

# ============================

l = len(min_two)
a = rows_with_mismatch_ids_null.sum()
print("\tArticles with mismatching IDs (incl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_mismatch_ids.sum()
print("\tArticles with mismatching IDs (excl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_match_ids.sum()
print("\tArticles with matching IDs: {} ({:.2f}%)".format(a, 100*a/l))

# ============================

res_ogids_match = results.loc[min_two[rows_with_match_ids].index]
mismatch_shares = res_ogids_match[shares].apply(lambda x: len(set([y for y in x if y!=0])) > 1, axis=1)
match_shares = res_ogids_match[shares].apply(lambda x: len(set([y for y in x if y!=0])) == 1 and len([y for y in x if y!=0])>1, axis=1)
has_shares = res_ogids_match[shares].apply(lambda x: len(set([y for y in x if y!=0])) < 1, axis=1)

l = len(res_ogids_match)
a = mismatch_shares.sum()
print("\t\tMatching IDs with mismatching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = match_shares.sum()
print("\t\tMatching IDs with matching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = has_shares.sum()
print("\t\tMatching IDs with no shares: {} ({:.2f}%)".format(a, 100*a/l))
print("Items with engagement but no shares: {}".format(len(res_ogids_match[has_shares])))

Total number of queried articles: 91490
Articles w/ at least 1 OG_ID:     26775 (29.27%)
Articles w/ at least 2 OG_IDs:    5007 (5.47%)
	Articles with mismatching IDs (incl Null): 5006 (99.98%)
	Articles with mismatching IDs (excl Null): 1910 (38.15%)
	Articles with matching IDs: 3097 (61.85%)
		Matching IDs with mismatching shares (excl. 0): 69 (2.23%)
		Matching IDs with matching shares (excl. 0): 635 (20.50%)
		Matching IDs with shares: 2306 (74.46%)
Items with engagement but no shares: 2306


In [104]:
print("Total number of queried articles WITH ENG > 0: {}".format(len(results_eng)))

# ============================
tdf = results_eng[ids].dropna(how="all")
print("Total at least one OG_ID: {}".format(len(tdf)))

# One OG_ID
x = tdf.apply(lambda x: x.notnull().sum() == 1, axis=1)
one = tdf[x]
print("One OG_ID    {}".format(len(one)))

# One OG_ID
x = tdf.apply(lambda x: x.notnull().sum() == 2, axis=1)
two = tdf[x]
print("Two OG_ID    {}".format(len(two)))

# One OG_ID
x = tdf.apply(lambda x: x.notnull().sum() == 3, axis=1)
three = tdf[x]
print("Three OG_ID  {}".format(len(three)))

# One OG_ID
x = tdf.apply(lambda x: x.notnull().sum() == 4, axis=1)
four = tdf[x]
print("Four OG_ID   {}".format(len(four)))

# ============================

Total number of queried articles WITH ENG > 0: 5498
Total at least one OG_ID: 5392
One OG_ID    3687
Two OG_ID    1535
Three OG_ID  161
Four OG_ID   9


In [204]:
def check_pairs(row):
    ids = ['1', '2', '3', '4']
    
    atleastonematching = False
    for c in itertools.combinations(ids, 2):
        x = c[0]
        y = c[1]
        
        # if one of the Ob_IDs is empty, move on
        if row['og_id' + x] is None or row['og_id' + y] is None:
            continue

        # keep track of matching Ob_IDs
        if row['og_id' + x] == row['og_id' + y]:
            atleastonematching = True

            # if we have matching IDs, but non matching values, it is a problem
            if row['eng' + x] != row['eng' + y]:
                return False
    
    if atleastonematching:
        return True

    return "nonmatch"

def check_nonmatching(row):
    ids = ['1', '2', '3', '4']
    
    for c in itertools.combinations(ids, 2):
        x = c[0]
        y = c[1]
        
        # if one of the Ob_IDs is empty, move on
        if row['og_id' + x] is None or row['og_id' + y] is None:
            continue
        
        if row['og_id' + x] != row['og_id' + y]:
            return True
    return False

In [572]:
df = two_eng

df['check_pairs'] = df.apply(check_pairs, axis=1)
x = df.groupby('check_pairs').size()
print(x)
print("nonmatch", df.apply(check_nonmatching, axis=1).sum())

True
check_pairs
False       146
True        620
nonmatch    769
dtype: int64
nonmatch 769


In [570]:
df = three_eng

df['check_pairs'] = df.apply(check_pairs, axis=1)
x = df.groupby('check_pairs').size()
print(x)
print("nonmatch", df.apply(check_nonmatching, axis=1).sum())

True
check_pairs
False       43
True        99
nonmatch    19
dtype: int64
nonmatch 131


In [568]:
df = four_eng

df['check_pairs'] = df.apply(check_pairs, axis=1)
x = df.groupby('check_pairs').size()
print(len(df) == x.sum())
print(x)

print("nonmatch", df.apply(check_nonmatching, axis=1).sum())

True
check_pairs
False    3
True     6
dtype: int64
nonmatch 8


In [466]:
three_eng = results.loc[three.index]
four_eng = results.loc[four.index]

nm_ids = three_eng[ids].apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 3, axis=1)
m_ids_m_eng = []
m_ids_nm_eng = []

matching = three_eng[ids].apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 2, axis=1)
for i, row in three_eng[matching][ids+eng].iterrows():
    y = row[ids].tolist()
    d = [x for x in y if y.count(x) > 1][0]
    indices = [i for i,x in enumerate(y) if x == d]
    if len(set(row[eng][['eng'+str(i+1) for i in indices]]))==1:
        m_ids_m_eng.append(True)
    else:
        m_ids_m_eng.append(False)

matching = three_eng[ids].apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 1, axis=1)
for i, row in three_eng[matching][ids+eng].iterrows():
    y = row[ids].tolist()
    d = [x for x in y if y.count(x) > 1][0]
    indices = [i for i,x in enumerate(y) if x == d]
    if len(set(row[eng][['eng'+str(i+1) for i in indices]]))==1:
        m_ids_m_eng.append(True)
    else:
        m_ids_m_eng.append(False)    
m_ids_nm_eng = [not x for x in m_ids_m_eng]



print("Not matching IDs        {}".format(nm_ids.sum()))
print("Matching IDs, NM Eng    {}".format(sum(m_ids_nm_eng)))
print("Matching IDs, M Eng     {}".format(sum(m_ids_m_eng)))
print("Check if SUM == TWO:    {}".format(nm_ids.sum()+sum(m_ids_m_eng)+sum(m_ids_nm_eng) == len(three)))

Not matching IDs        19
Matching IDs, NM Eng    31
Matching IDs, M Eng     81
Check if SUM == TWO:    False


In [ ]:
x = results[eng].apply(lambda x: sum(x) > 0, axis=1)
tdf = results[x]
print("Total number of queried articles WITH ENG > 0: {}".format(len(tdf)))

# ============================
tdf = tdf[ids].dropna(how="all")

# One OG_ID
x = tdf.apply(lambda x: x.notnull().sum() == 1, axis=1)
one = tdf[x]

l = len(tdf)
a = len(min_one)
print("Articles w/ at least 1 OG_ID:     {} ({:.2f}%)".format(a, 100*a/l))
a = len(min_two)
print("Articles w/ at least 2 OG_IDs:    {} ({:.2f}%)".format(a, 100*a/l))

# ============================

rows_with_mismatch_ids_null = min_two.apply(lambda x: len(set(x)) != 1, axis=1)
rows_with_mismatch_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) != 1, axis=1)
rows_with_match_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 1 and len([y for y in x if pd.notnull(y)])>1, axis=1)

l = len(min_two)
a = rows_with_mismatch_ids_null.sum()
print("\tArticles with mismatching IDs (incl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_mismatch_ids.sum()
print("\tArticles with mismatching IDs (excl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_match_ids.sum()
print("\tArticles with matching IDs: {} ({:.2f}%)".format(a, 100*a/l))

# ============================

res_ogids_match = results.loc[min_two[rows_with_match_ids].index]
mismatch_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) > 1, axis=1)
match_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) == 1 and len([y for y in x if y!=0])>1, axis=1)
has_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) < 1, axis=1)

l = len(res_ogids_match)
a = mismatch_shares.sum()
print("\t\tMatching IDs with mismatching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = match_shares.sum()
print("\t\tMatching IDs with matching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = has_shares.sum()
print("\t\tMatching IDs with no shares: {} ({:.2f}%)".format(a, 100*a/l))
print("Items with engagement but no shares: {}".format(len(res_ogids_match[has_shares])))

In [389]:
tdf = results[eng].apply(lambda x: sum(x) > 0, axis=1)
tdf = results[tdf]
print("Total number of queried articles WITH ENG > 0: {}".format(len(tdf)))

# ============================

# At least one OG_ID
min_one = tdf[ids].dropna(how="all")
min_two_bool = min_one.apply(lambda x: x.notnull().sum() > 1, axis=1)
min_two = min_one[min_two_bool]

l = len(tdf)
a = len(min_one)
print("Articles w/ at least 1 OG_ID:     {} ({:.2f}%)".format(a, 100*a/l))
a = len(min_two)
print("Articles w/ at least 2 OG_IDs:    {} ({:.2f}%)".format(a, 100*a/l))

# ============================

rows_with_mismatch_ids_null = min_two.apply(lambda x: len(set(x)) != 1, axis=1)
rows_with_mismatch_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) != 1, axis=1)
rows_with_match_ids = min_two.apply(lambda x: len(set([y for y in x if pd.notnull(y)])) == 1 and len([y for y in x if pd.notnull(y)])>1, axis=1)

l = len(min_two)
a = rows_with_mismatch_ids_null.sum()
print("\tArticles with mismatching IDs (incl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_mismatch_ids.sum()
print("\tArticles with mismatching IDs (excl Null): {} ({:.2f}%)".format(a, 100*a/l))
a = rows_with_match_ids.sum()
print("\tArticles with matching IDs: {} ({:.2f}%)".format(a, 100*a/l))

# ============================

res_ogids_match = results.loc[min_two[rows_with_match_ids].index]
mismatch_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) > 1, axis=1)
match_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) == 1 and len([y for y in x if y!=0])>1, axis=1)
has_shares = res_ogids_match[eng].apply(lambda x: len(set([y for y in x if y!=0])) < 1, axis=1)

l = len(res_ogids_match)
a = mismatch_shares.sum()
print("\t\tMatching IDs with mismatching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = match_shares.sum()
print("\t\tMatching IDs with matching shares (excl. 0): {} ({:.2f}%)".format(a, 100*a/l))
a = has_shares.sum()
print("\t\tMatching IDs with no shares: {} ({:.2f}%)".format(a, 100*a/l))
print("Items with engagement but no shares: {}".format(len(res_ogids_match[has_shares])))

Total number of queried articles WITH ENG > 0: 5498
Articles w/ at least 1 OG_ID:     5392 (98.07%)
Articles w/ at least 2 OG_IDs:    1705 (31.01%)
	Articles with mismatching IDs (incl Null): 1704 (99.94%)
	Articles with mismatching IDs (excl Null): 908 (53.26%)
	Articles with matching IDs: 797 (46.74%)
		Matching IDs with mismatching shares (excl. 0): 72 (9.03%)
		Matching IDs with matching shares (excl. 0): 638 (80.05%)
		Matching IDs with no shares: 0 (0.00%)
Items with engagement but no shares: 0


## Problem 3 - URLs collapse into same graph object

In [94]:
df = results[ids]
ids = ['og_id%s' % i for i in range(1,5)]

df['all_ids'] = df[ids].apply(lambda x: [int(y) for y in set(x) if pd.notnull(y)], axis=1)

all_ids = df.all_ids.sum()

counter = collections.Counter(all_ids)
dup_ids = set([i for (i,v) in counter.items() if v > 1])

print("Number of duplicate ids: %s" % len(dup_ids))

df['has_dup'] = df.all_ids.map(lambda x: len(dup_ids.intersection(x)) > 0)
print("Number of articles affected: %s" % df.has_dup.sum())

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Number of duplicate ids: 66
Number of articles affected: 507


/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [132]:
df = results[ids].reset_index(drop=True)

dupls = [False] * len(df)
seen_ids = set()
x = {}

arr = df.values
a = arr[arr != np.array(None)].ravel()
q = np.unique(a, return_counts=True)
dup_ids = q[0][q[1] > 1]

f = lambda x: set(np.where(arr == x)[0])
fv = np.vectorize(f)
c = fv(dup_ids)
    
seen_ids = set()
for row in tqdm(df.itertuples(), total=len(df)):
    for val in [x for x in set(row[1:5]) if x is not None]:
        if val not in seen_ids:
            try:
                indices = c[dup_ids==val]
                if len(indices)>1:
                    seen_ids.add(val)
                    for i in indices:
                        dupls[i] = True
            except:
                pass
sum(dupls)

100%|██████████| 91490/91490 [00:02<00:00, 31565.48it/s]


0

In [21]:
x = df[dupls].apply(pd.value_counts).sum(axis=1)
x[x>1].sort_values(ascending=False).reset_index()

,index,0
0,1649721495056000,38.0
1,1382544005159501,10.0
2,926022794172969,8.0
3,1540865556012732,4.0
4,10150284009700488,4.0
5,947150088698533,3.0
6,1827602397257489,3.0
7,566483340123620,2.0
8,456681777755197,2.0
9,1826664834073265,2.0


# Interesting DOIs

**10.1007/s00586-013-2675-y**

Same OG ID for http/https but different one for DOI. Different share numbers

**10.1038/nature13893**

Different OG objects

**10.7717/peerj.794**

Same share numbers, different OG IDs

**10.1016/j.aap.2014.03.007**

Elsevier redirect page, various OG IDs

**10.7440/res53.2015.10**

Various IDs across DOI, URL

## Used code

```
doi = '10.7440/res53.2015.10'
rec = df.loc[doi]
urls = [rec.url, rec.url2, "https://doi.org/%s" % doi, "http://dx.doi.org/%s" % doi]
pprint(fb_queries(urls))
```

## Problem with missing scraped data

Results for the http and https for journals.ametsoc.org/doi/abs/10.1175/JAS-D-12-0315.1 

URL    | OG ID            | Shares | Date (scrape)
-------|------------------|--------|---------------
HTTP   | 685490234864647  | 2      | September 30, 2016
HTTPS  | None             | None   | None

After manually triggering a rescrape:

URL    | OG ID            | Shares | Date (scrape)
-------|------------------|--------|---------------
HTTP   | 1818366768210382 | 0      | March 28, 2018
HTTPS  | 1818366768210382 | 0      | March 28, 2018

Apparently shares associated with previous canonical URLs are lost...